In [1]:
import gradio as gr
import torch
import torch.nn as nn
from torchvision.models import resnet50
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Define the model architecture (same as during training)
class ModifiedResNet(nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        self.resnet = resnet50(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(
            in_channels=1,              # Single channel for grayscale
            out_channels=64,
            kernel_size=(7, 7),
            stride=(2, 2),
            padding=(3, 3),
            bias=False
        )
        self.resnet.fc = nn.Linear(
            in_features=self.resnet.fc.in_features,
            out_features=14               # Number of classes
        )
    
    def forward(self, x):
        return self.resnet(x)

# Initialize the model
model = ModifiedResNet()

# Load the saved weights
model.load_state_dict(torch.load('resnet50_scratch.pth'))
model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Move the model to GPU if available
model.eval()      # Set the model to evaluation mode

# Define the image transformations for grayscale images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Function to predict on a single image
def predict(image):
    # Load and preprocess the image
    image = Image.fromarray(np.array(image).astype('uint8')).convert('L')  # Convert to grayscale
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension

    # Move image to GPU if available
    image = image.to('cuda' if torch.cuda.is_available() else 'cpu')

    # Perform prediction
    with torch.no_grad():
        outputs = model(image)
        predictions = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
        predicted_labels = (predictions > 0.15).float()
    
    # Convert the tensor to a list of predicted pathologies
    labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 
              'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Pleural Thickening',
              'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']
    return {label: bool(pred) for label, pred in zip(labels, predicted_labels[0].tolist())}




c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Matt\AppData\Local\Temp\ipykernel_7464\2729737196.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the defau

In [2]:
# Create the Gradio interface
iface = gr.Interface(fn=predict, inputs=gr.Image(type='pil'), outputs=gr.Label(num_top_classes=14))

# Launch the Gradio app
iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
